In [8]:
from db.helpers import gm_sales_collection
import pandas as pd
from datetime import datetime
from dateutil.relativedelta import relativedelta

In [9]:
keys = ['Weekday_Store_Sales','Weekday_Delivery_Sales','Weekend_Store_Sales','Weekend_Delivery_Sales']

def calculate_growth(value1, value2):
    if value1 == 0:
        return None 
    else:
        growth = ((value2 - value1) / value1) 
        return growth

def group_sales(group_id,match):
    return gm_sales_collection.aggregate([
        {'$match': {**match,"Level_1_Area":"Kuwait"}},
    {
        '$group': {
            '_id': {**group_id,"year":"$Sales_Year","month":"$Sales_Month"}, 
            'Weekday_Store_Sales': {
                '$sum': '$Weekday_Store_Sales'
            }, 
            'Weekday_Delivery_Sales': {
                '$sum': '$Weekday_Delivery_Sales'
            }, 
            'Weekend_Store_Sales': {
                '$sum': '$Weekend_Store_Sales'
            }, 
            'Weekend_Delivery_Sales': {
                '$sum': '$Weekend_Delivery_Sales'
            },
                "numberOfOutlets":{"$sum":1}
        }
    },
    {   "$sort":{
            "_id.year":1,
             "_id.month":1
        }
    },
])

def generate_seasonality_record(base,data):
    result = {**base}
    keys = ['Weekday_Store_Sales','Weekday_Delivery_Sales','Weekend_Store_Sales','Weekend_Delivery_Sales']
    for key in keys: 
        if len(data) != 2:
            result[key] = None
            continue
        growth = calculate_growth(data[0][key]/data[0]['numberOfOutlets'],data[1][key]/data[1]['numberOfOutlets'])
        if growth:
            if (growth < 2 and growth > -1):
                result[key] = growth
                continue
    return result


def getDates(start_date:datetime=datetime(2016, 1, 1),end_date:datetime=datetime(2023, 12, 1)):
    date = start_date
    while date < end_date:
        yield date
        date += relativedelta(months=1)

def filter_sales(data:list,date_1:datetime,date_2:datetime):
    return [record for record in data if (record['_id']['year'] == date_1.year and record['_id']['month'] == date_1.month) or (record['_id']['year'] == date_2.year  and record['_id']['month']== date_2.month)]

In [10]:
# remove for loop for years and moths and use getDates 

generate Location Type Seasonality

In [17]:
location_types = gm_sales_collection.distinct("Location_Type",{"Location_Type":{"$ne":0}})
_id = {'Location_type': '$Location_Type'}
result = []
for i in location_types:
    for date in getDates():
        last_month = (date - relativedelta(months=1))
        data = filter_sales(list(group_sales(_id,{"Location_Type":i,"Sales_Month":{"$in":[date.month,last_month.month]},"Sales_Year":{"$in":[date.year,last_month.year]}})),date,last_month)
        result.append(generate_seasonality_record({"location_type":i,"year":date.year,"month":date.month},data))

for record in result:
    for key in keys:
            if key not in record:
                current_date = datetime(record['year'],record['month'],1)

                last_month = current_date - relativedelta(months=1)
                all_locations_growth = filter_sales(list(group_sales({},{"Sales_Month":{"$in":[current_date.month,last_month.month]},"Sales_Year":{"$in":[current_date.year,last_month.year]}})),current_date,last_month)
                if len(all_locations_growth) != 2:
                    raise Exception("all_locations_growth length issue")
                if current_date.year == 2021 and current_date.month == 12:
                    print(all_locations_growth)
                first_month = all_locations_growth[0][key]/all_locations_growth[0]['numberOfOutlets']
                second_month = all_locations_growth[1][key]/all_locations_growth[1]['numberOfOutlets']
                growth = calculate_growth(first_month,second_month)
                # Check next month, add all_locations_growth to next month growth
                record[key] = growth
location_type_df = pd.DataFrame(result)
location_type_df.to_csv("seasonality.csv",index=False)

generate Products Seasonality

In [12]:
products_types = gm_sales_collection.distinct("Product_Focus",{"Level_1_Area":"Kuwait","Product_Focus":{"$ne":0}})
_id = {'Product_Focus': '$Product_Focus'}
result = []
for i in products_types:
    for date in getDates():
        last_month = (date - relativedelta(months=1))
        data = filter_sales(list(group_sales(_id,{"Product_Focus":i,"Sales_Month":{"$in":[date.month,last_month.month]},"Sales_Year":{"$in":[date.year,last_month.year]}})),date,last_month)
        result.append(generate_seasonality_record({"product_focus":i,"year":date.year,"month":date.month},data))
for record in result:
    for key in keys:
            if key not in record:
                current_date = datetime(record['year'],record['month'],1)
                last_month = current_date - relativedelta(months=1)
                all_locations_growth = filter_sales(list(group_sales({},{"Sales_Month":{"$in":[current_date.month,last_month.month]},"Sales_Year":{"$in":[current_date.year,last_month.year]}})),current_date,last_month)
                if len(all_locations_growth) != 2:
                    # raise Exception("all_locations_growth length issue")
                    continue
                first_month = all_locations_growth[0][key]/all_locations_growth[0]['numberOfOutlets']
                second_month = all_locations_growth[1][key]/all_locations_growth[1]['numberOfOutlets']
                growth = calculate_growth(first_month,second_month)
                # Check next month, add all_locations_growth to next month growth
                record[key] = growth
product_focus_df = pd.DataFrame(result)

generate Area Seasonality

In [13]:
areas = gm_sales_collection.distinct("Level_3_Area",{"Level_1_Area":"Kuwait"})
_id = {'Level_3_Area': '$Level_3_Area'}
result = []
for i in areas:
    for date in getDates():
        last_month = (date - relativedelta(months=1))
        data = filter_sales(list(group_sales(_id,{"Level_3_Area":i,"Sales_Month":{"$in":[date.month,last_month.month]},"Sales_Year":{"$in":[date.year,last_month.year]}})),date,last_month)
        result.append(generate_seasonality_record({"area":i,"year":date.year,"month":date.month},data))
# check growth for level 2 area
for record in result:
    for key in keys:
            if key not in record:
                current_date = datetime(record['year'],record['month'],1)
                last_month = current_date - relativedelta(months=1)
                all_locations_growth = filter_sales(list(group_sales({'Level_2_Area':"$Level_2_Area"},{"Sales_Month":{"$in":[current_date.month,last_month.month]},"Sales_Year":{"$in":[current_date.year,last_month.year]}})),current_date,last_month)
                if len(all_locations_growth) != 2:
                    continue
                    # raise Exception("all_locations_growth length issue")
                first_month = all_locations_growth[0][key]/all_locations_growth[0]['numberOfOutlets']
                second_month = all_locations_growth[1][key]/all_locations_growth[1]['numberOfOutlets']
                growth = calculate_growth(first_month,second_month)
                # Check next month, add all_locations_growth to next month growth
                record[key] = growth
area_df = pd.DataFrame(result)

generate Industry Type Seasonality

In [14]:
industry = gm_sales_collection.distinct("Industry_Level_2",{"Level_1_Area":"Kuwait","Industry_Level_2":{"$ne":0}})
_id = {'Industry_Level_2': 'Industry_Level_2'}
result = []

def group_sales_2(group_id,match,industry):
    pipeline =[
        {'$match': {**match,"Level_1_Area":"Kuwait"}},
        {"$lookup":{
  "from": "Brands",
  "localField": "Brand",
  "foreignField": "Brand_Name_English",
  "as": "brand",
  "pipeline": [
    {
      "$match": {
        "Industry_Level_1": industry,
      },
    },
  ],
}},
{"$match":{"brand.0":{"$exists":True}}},
    {
        '$group': {
            '_id': {**group_id,"year":"$Sales_Year","month":"$Sales_Month"}, 
            'Weekday_Store_Sales': {
                '$sum': '$Weekday_Store_Sales'
            }, 
            'Weekday_Delivery_Sales': {
                '$sum': '$Weekday_Delivery_Sales'
            }, 
            'Weekend_Store_Sales': {
                '$sum': '$Weekend_Store_Sales'
            }, 
            'Weekend_Delivery_Sales': {
                '$sum': '$Weekend_Delivery_Sales'
            },
                "numberOfOutlets":{"$sum":1}
        }
    },
    {   "$sort":{
            "_id.year":1,
             "_id.month":1
        }
    },
]
    return gm_sales_collection.aggregate(pipeline)

industry_table = {
  "Events": "Foodservice",
  "Jewellery": "Service",
  "Fast Food": "Foodservice",
  "Electronics": "Service",
  "Entertainment": "Retail",
  "Home Appliance": "Products",
  "Professional Services": "Office",
  "Cleaning": "Service",
  "Accessories": "Service",
  "Super Market": "Retail",
  "Construction Material": "Service",
  "Hospitality": "Retail",
  "Music": "Retail",
  "Livestock": "Service",
  "Furniture": "Factory",
  "Supermarket": "Retail",
  "Printing": "Service",
  "Toys": "Retail",
  "Parking": "Service",
  "Home Improvements": "Retail",
  "Coffeeshop": "Foodservice",
  "Construction": "Service",
  "Full Service Restaurant": "Foodservice",
  "Novelties": "Retail",
  "Bars": "Factory",
  "Food Truck": "Foodservice",
  "Government Services": "Service",
  "Hypermarket": "Retail",
  "Textile": "Retail",
  "Communication Materials and Systems": "Retail",
  "100% Home Delivery": "Foodservice",
  "undefined": "Foodservice",
  "Vocational Services": "Service",
  "Personal Care": "Service",
  "Warehouse": "Service",
  "Catering": "Foodservice",
  "Department Store": "Retail",
  "Media": "Service",
  "Sports": "Retail",
  "Textiles": "Retail",
  "Bakery": "Foodservice",
  "Hunting": "Retail",
  "Security": "Service",
  "Sport Goods": "Retail",
  "Logistics": "Products",
  "Plumbing Services": "Service",
  "Gifts": "Retail",
  "E-Commerce": "Retail",
  "Bookstore": "Retail",
  "Equipment": "Retail",
  "Cooperative Branch": "Retail",
  "Motors": "Products",
  "Pets": "Service",
  "Weapons": "Retail",
  "General Store": "Retail",
  "Gas Station": "Retail",
  "FMCG": "Foodservice",
  "Vending Machines": "Retail",
  "Repair": "Service",
  "Convenience Store": "Foodservice",
  "Food Retail": "Retail",
  "Government": "Service",
  "Cooperative": "Retail",
  "Training": "Service",
  "Photography": "Service",
  "Telecom": "Service",
  "Oil & Gas": "Service",
  "Agricultural & Industrial Machines": "Retail",
  "Coworking Space": "Service",
  "Contracting": "Service",
  "Cooperative Wholesale": "Retail",
  "Healthcare": "Retail",
  "Discount Retail": "Retail",
  "Religious Services": "Service",
  "Food Hall": "Foodservice",
  "Fast Casual": "Foodservice",
  "Smoking": "Products",
  "Cafe": "Foodservice",
  "Staffing": "Service",
  "Buffet": "Foodservice",
  "Technology": "Service",
  "Farming": "Retail",
  "Laundry": "Service",
  "Apparel": "Retail",
  "Property Management": "Service",
  "Tailor": "Service",
  "Instant Offices": "Service",
  "Health Club": "Service",
  "Travel": "Retail",
  "Car Rental": "Service",
  "Footwear": "Retail",
  "Laundary": "Service",
  "Financial Services": "Service",
  "Postal Office": "Service",
  "Wholesale": "Retail",
  "Exhibition": "Service",
  "Central Kitchen": "Foodservice",
  "Pharmacy": "Retail",
  "bakery": "Foodservice",
  "Stationary": "Service",
  "E-commerce": "Retail",
  "IT": "Service",
  "Scientific Instrument": "Retail",
  "Mini Market": "Retail",
  "Education": "Service",
  "Art": "Retail",
  "Taxi": "Service",
  "Community Services": "Service"
}

for i in industry:
    for date in getDates():
        last_month = (date - relativedelta(months=1))
        data = filter_sales(list(group_sales(_id,{"Industry_Level_2":i,"Sales_Month":{"$in":[date.month,last_month.month]},"Sales_Year":{"$in":[date.year,last_month.year]}})),date,last_month)
        result.append(generate_seasonality_record({"industry":i,"year":date.year,"month":date.month},data))

for record in result:
    for key in keys:
            if key not in record:
                industry =industry_table[record['industry']]
                current_date = datetime(record['year'],record['month'],1)
                last_month = current_date - relativedelta(months=1)
                all_locations_growth = filter_sales(list(group_sales_2({},{"Sales_Month":{"$in":[current_date.month,last_month.month]},"Sales_Year":{"$in":[current_date.year,last_month.year]}},industry)),current_date,last_month)
                if len(all_locations_growth) != 2:
                    continue
                    # raise Exception("all_locations_growth length issue")
                first_month = all_locations_growth[0][key]/all_locations_growth[0]['numberOfOutlets']
                second_month = all_locations_growth[1][key]/all_locations_growth[1]['numberOfOutlets']
                growth = calculate_growth(first_month,second_month)
                # Check next month, add all_locations_growth to next month growth
                record[key] = growth
industry_df = pd.DataFrame(result)

In [16]:
with pd.ExcelWriter('multiple.xlsx', engine='xlsxwriter') as writer:
    location_type_df.to_excel(writer, sheet_name='location_type')
    product_focus_df.to_excel(writer, sheet_name='product_focus')
    area_df.to_excel(writer, sheet_name='area')
    industry_df.to_excel(writer, sheet_name='industry')
